# Transportation

$$
\begin{align}
    \text{min} \quad & \sum_{i \in I}\sum_{j \in J} c_{i, j} x_{i, j} \\
    \text{s.t.} \quad & \sum_{j \in J} x_{i, j} \leq b_{i} & \forall \; i \in I \\
    \quad & \sum_{i \in I} x_{i, j} = d_{j} & \forall \; j \in J \\
    & x_{i, j} \geq 0 & \forall \;i \in I, j \in J \\
\end{align}
$$

In [1]:
# Python native modules
import json

# Third-party packages
import pandas as pd  # Tabular data
import pyomo.environ as pyo

In [2]:
# Read input file and store in local variable `input_data`
with open("input_transp.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [3]:
# Create local variables to store input parameters
availabilities = input_data["availabilities"]
demands = input_data["demands"]

# Dictionary of costs indexed by tuples (origin, destination)
costs = {
    (c["from"], c["to"]): c["value"]
    for c in input_data["costs"]
}

In [6]:
print('availabilities:',availabilities)
print('demands:',demands)
print('costs:',costs)

availabilities: {'S1': 14, 'S2': 26, 'S3': 11}
demands: {'C1': 5, 'C2': 13, 'C3': 15, 'C4': 17}
costs: {('S1', 'C1'): 10, ('S1', 'C2'): 5, ('S1', 'C3'): 20, ('S1', 'C4'): 12, ('S2', 'C1'): 12, ('S2', 'C2'): 7, ('S2', 'C3'): 12, ('S2', 'C4'): 19, ('S3', 'C1'): 6, ('S3', 'C2'): 12, ('S3', 'C3'): 16, ('S3', 'C4'): 17}


In [7]:
# Create model instance
model = pyo.ConcreteModel()

In [8]:
# Sets for suppliers I and customers J
model.I = pyo.Set(initialize=availabilities.keys())
model.J = pyo.Set(initialize=demands.keys())

In [9]:
# Paramters
model.b = pyo.Param(model.I, initialize=availabilities)
model.d = pyo.Param(model.J, initialize=demands)
model.c = pyo.Param(model.I, model.J, initialize=costs)

In [10]:
# Decision variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)

In [11]:
# Supplier availablity constraints
def av_cstr(model, i):
    return sum(model.x[i,j] for j in model.J) <= model.b[i]
    # return sum(model.x[i,:]) <= model.b[i]

model.av_cstr = pyo.Constraint(model.I, rule=av_cstr)

In [22]:
model.av_cstr.pprint()

av_cstr : Size=3, Index=I, Active=True
    Key : Lower : Body                                      : Upper : Active
     S1 :  -Inf : x[S1,C1] + x[S1,C2] + x[S1,C3] + x[S1,C4] :  14.0 :   True
     S2 :  -Inf : x[S2,C1] + x[S2,C2] + x[S2,C3] + x[S2,C4] :  26.0 :   True
     S3 :  -Inf : x[S3,C1] + x[S3,C2] + x[S3,C3] + x[S3,C4] :  11.0 :   True


In [15]:
# Demand equality constraints
def dem_cstr(model, j):
    return sum(model.x[i,j] for i in model.I) == model.d[j]
    # return sum(model.x[:,j]) == model.d[j]

model.dem_cstr = pyo.Constraint(model.J, rule=dem_cstr)

In [23]:
model.dem_cstr.pprint()

dem_cstr : Size=4, Index=J, Active=True
    Key : Lower : Body                           : Upper : Active
     C1 :   5.0 : x[S1,C1] + x[S2,C1] + x[S3,C1] :   5.0 :   True
     C2 :  13.0 : x[S1,C2] + x[S2,C2] + x[S3,C2] :  13.0 :   True
     C3 :  15.0 : x[S1,C3] + x[S2,C3] + x[S3,C3] :  15.0 :   True
     C4 :  17.0 : x[S1,C4] + x[S2,C4] + x[S3,C4] :  17.0 :   True


In [16]:
# Objective function
def obj(model):
    total_cost = sum(
        model.x[i,j] * model.c[i,j]
        for i in model.I
        for j in model.J
    )
    return total_cost


model.obj = pyo.Objective(rule=obj, sense=pyo.minimize)

In [17]:
# Instantiate Highs persistent solver (make sure highspy is installed)
solver = pyo.SolverFactory("appsi_highs")

In [18]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 526.0, 'Upper bound': 526.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [19]:
# Use objective as a callable to see its value
model.obj()

526.0

In [20]:
sol = [
    {"from": i, "to": j, "value": val}
    for (i, j), val in model.x.extract_values().items()
]

In [21]:
dataframe = pd.DataFrame(sol).pivot(
    index="from", columns="to", values="value"
)

In [36]:
display(dataframe)

to,C1,C2,C3,C4
from,,,,
S1,0.0,2.0,0.0,12.0
S2,0.0,11.0,15.0,0.0
S3,5.0,0.0,0.0,5.0
